<a href="https://colab.research.google.com/github/yongjuncho123/dust/blob/main/job04_API_%ED%99%9C%EC%9A%A9_%EB%B0%8F_%EC%8B%A4%EC%A0%9C_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A1%9C_%EC%A0%90%EA%B2%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xmltodict

In [ ]:
import pandas as pd
import requests
import xmltodict
import xml.etree.ElementTree as ET
from tensorflow.keras.models import load_model
import time
from datetime import datetime
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data/project02_dust/test_Data/서울시 대기질 자료 제공_2020-2021.csv', encoding = 'cp949')

df = df[df['구분'] == '강동구'].loc[::-1].reset_index(drop = True)

df

,일시,구분,미세먼지(PM10),초미세먼지(PM25)
0,2020-01-01 0:00,강동구,27.0,18.0
1,2020-01-01 1:00,강동구,25.0,19.0
2,2020-01-01 2:00,강동구,25.0,18.0
3,2020-01-01 3:00,강동구,25.0,19.0
4,2020-01-01 4:00,강동구,24.0,18.0
...,...,...,...,...
17539,2021-12-31 19:00,강동구,25.0,8.0
17540,2021-12-31 20:00,강동구,25.0,8.0
17541,2021-12-31 21:00,강동구,28.0,10.0
17542,2021-12-31 22:00,강동구,25.0,8.0


In [ ]:
model_pm10 = load_model('/content/drive/MyDrive/data/project02_dust/LSTM 모델/LSTM_PM10.h5')
model_pm25 = load_model('/content/drive/MyDrive/data/project02_dust/LSTM 모델/LSTM_PM25.h5')

In [ ]:
apikey = '61466878736a6f653130385974494949'
region = '강동구'
boarderline = '동남권'
startnum = 1
endnum = 1000

In [ ]:
url = f'http://openAPI.seoul.go.kr:8088/{apikey}/xml/RealtimeCityAir/{startnum}/{endnum}/{boarderline}/{region}'

In [ ]:
response = requests.get(url)
print(response.content)

b'<?xml version="1.0" encoding="UTF-8"?>\n<RealtimeCityAir>\n<list_total_count>1</list_total_count>\n<RESULT>\n<CODE>INFO-000</CODE>\n<MESSAGE>\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4</MESSAGE>\n</RESULT>\n<row>\n<MSRDT>202409051800</MSRDT>\n<MSRRGN_NM>\xeb\x8f\x99\xeb\x82\xa8\xea\xb6\x8c</MSRRGN_NM>\n<MSRSTE_NM>\xea\xb0\x95\xeb\x8f\x99\xea\xb5\xac</MSRSTE_NM>\n<PM10>15</PM10>\n<PM25>12</PM25>\n<O3>0.037</O3>\n<NO2>0.014</NO2>\n<CO>0.5</CO>\n<SO2>0.002</SO2>\n<IDEX_NM>\xeb\xb3\xb4\xed\x86\xb5</IDEX_NM>\n<IDEX_MVL>56</IDEX_MVL>\n<ARPLT_MAIN>O3</ARPLT_MAIN>\n</row>\n</RealtimeCityAir>\n'


In [ ]:
print(response.status_code)

200


In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import os

# API 요청 인자 설정
KEY = 'yourkey'  # 인증키
TYPE = 'xml'  # 요청 파일 타입
SERVICE = 'ListAirQualityByDistrictService'  # 서비스명
START_INDEX = '1'  # 요청 시작 위치
END_INDEX = '5'  # 요청 종료 위치
MSRADMCODE = '111274'  # 측정소 행정코드 (예: 강동구)

# API 요청 URL
url = f"http://openAPI.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START_INDEX}/{END_INDEX}/{MSRADMCODE}/"

# API 호출
response = requests.get(url)

# XML 데이터 파싱
root = ET.fromstring(response.content)

# 새로운 데이터 리스트 초기화
new_data = []

for item in root.findall('.//row'):
    msrdate = item.find('MSRDATE').text if item.find('MSRDATE') is not None else None
    station_name = item.find('MSRSTENAME').text if item.find('MSRSTENAME') is not None else None
    pm10 = item.find('PM10').text if item.find('PM10') is not None else None
    pm25 = item.find('PM25').text if item.find('PM25') is not None else None
    new_data.append([msrdate, station_name, pm10, pm25])

# 새로운 데이터를 DataFrame으로 변환
new_df = pd.DataFrame(new_data, columns=['측정시간', '측정소명', 'PM10 농도', 'PM25 농도'])



# 기존 데이터프레임이 존재하는지 확인
if os.path.exists('/content/drive/MyDrive/data/project02_dust/24_data/air_quality.csv'):
    # 기존 CSV 파일을 읽어옴
    existing_df = pd.read_csv('/content/drive/MyDrive/data/project02_dust/24_data/air_quality.csv')

    # 기존 데이터프레임에 새로운 데이터를 추가 (중복 제거)
    combined_df = pd.concat([existing_df, new_df]).drop_duplicates().reset_index(drop=True)
else:
    # 기존 데이터가 없으면 새로운 데이터만 사용

    combined_df = new_df

# 마지막으로 들어온 데이터가 '점검중'이면 앞선 1시간 데이터로 계산
if combined_df['PM10 농도'].iloc[-1] == '점검중':
    combined_df['PM10 농도'].iloc[-1] = combined_df['PM10 농도'].iloc[-3]


# 업데이트된 데이터프레임을 CSV 파일로 저장
combined_df.to_csv('/content/drive/MyDrive/data/project02_dust/24_data/air_quality.csv', index = False, encoding = 'utf-8')

# CSV 파일 읽기 및 출력
df = pd.read_csv('/content/drive/MyDrive/data/project02_dust/24_data/air_quality.csv')
print(df)


            측정시간 측정소명  PM10 농도  PM25 농도
0   202409050600  강동구       18       13
1   202409050700  강동구       22       12
2   202409050800  강동구       21       10
3   202409050900  강동구       24       13
4   202409051000  강동구       34       16
5   202409051200  강동구       35       13
6   202409051300  강동구       20        9
7   202409051400  강동구       17       11
8   202409051500  강동구       17       17
9   202409051600  강동구       14       14
10  202409051800  강동구       15       12
11  202409051900  강동구       27       16
12  202409052000  강동구       26       17
13  202409052100  강동구       30       14


In [ ]:
# PM10 데이터만 추출 (새로운 데이터로 업데이트된 df 사용)
pm10_data = df['PM10 농도'].values

# MinMaxScaler 객체 생성 및 적용
scaler = MinMaxScaler()
pm10_data_scaled = scaler.fit_transform(pm10_data.reshape(-1, 1))

# 시퀀스를 생성하는 함수 (위의 코드와 동일)
def sequenceCut(data, len_step):
    sequence = []  # 시퀀스 덩어리 리스트
    for i in range(len(data) - len_step):
        sequence.append(data[i : i + len_step])  # 시퀀스 생성
    return np.array(sequence)

# 시퀀스 길이 설정
len_step = 5

# 시퀀스 생성 (스케일링된 데이터 사용)
X = sequenceCut(pm10_data_scaled, len_step)

In [ ]:
# 모델 예측 (마지막 len_step 시퀀스를 입력으로 사용하여 1시간 앞의 PM10 농도 예측)
y_pred_scaled = model_pm10.predict(X[-1].reshape(1, len_step, 1))  # LSTM에 맞게 reshape

# 예측된 값을 원래 스케일로 복원
y_pred_original = scaler.inverse_transform(y_pred_scaled)

# 예측값 출력
print(f"예측된 PM10 농도: {y_pred_original[0][0]}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
예측된 PM10 농도: 21.53917121887207


In [ ]:
print(y_pred_original[0][0])

answer = pd.DataFrame({'pred' : [y_pred_original[0][0]]}).reset_index(drop = True)



print(answer)
answer.to_csv('/content/drive/MyDrive/data/project02_dust/24_data/pred.csv')

# CSV 파일 경로
csv_path = '/content/drive/MyDrive/data/project02_dust/24_data/pred.csv'

# 기존 CSV 파일이 존재하는지 확인
if os.path.exists(csv_path):
    # 기존 CSV 파일을 읽어옴
    existing_answer = pd.read_csv(csv_path, index_col=0)

    # 기존 데이터프레임에 새로운 데이터를 추가 (중복 제거)
    combined_answer = pd.concat([existing_answer, answer]).drop_duplicates().reset_index(drop = True)
else:
    # 기존 파일이 없으면 새로운 데이터만 사용
    combined_answer = answer

# CSV 파일에 덮어쓰기 하지 않도록 새로운 데이터를 병합한 결과를 저장
combined_answer.to_csv(csv_path, index = False)  # 덮어쓰기 방지를 위해 'index=False' 사용

21.539171
        pred
0  21.539171


In [ ]:
answer['pred'].loc[0]


17.160187

In [ ]:
a = answer['pred'].iloc[-1]

if a < 31:
    print(f'한 시간 뒤 예측 미세먼지 농도는 좋음입니다! 예측 농도: {a:.2f}' )
elif a < 81:
    print(f'한 시간 뒤 예측 미세먼지 농도는 보통입니다! 예측 농도: {a:.2f}' )
elif a < 151:
    print(f'한 시간 뒤 예측 미세먼지 농도는 나쁨입니다ㅠ 예측 농도: {a:.2f}' )
else:
    print(f'한 시간 뒤 예측 미세먼지 농도는 나쁨입니다ㅠㅠㅠ 예측 농도: {a:.2f}' )

한 시간 뒤 예측 미세먼지 농도는 좋음입니다! 예측 농도: 21.54
